In [1]:
%cd /Users/maayanlab/Documents/projects/human-melanoma-signature
import sys
sys.path.append('pipeline/scripts/')
import pandas as pd
import h5py
import numpy as np
import json, operator, math
from geode_jupies import *

/Users/maayanlab/Documents/projects/human-melanoma-signature


/Users/maayanlab/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
%pwd

'/Users/maayanlab/Documents/projects/human-melanoma-signature'

In [3]:
from Melanoma import *

# Code Development for Melanoma Pipeline
## Define input and output files

In [4]:
#infiles
sig = 's2-signatures.dir/melanoma-signatures.txt'

In [5]:
# #infiles
# cpm = 's1-expression_data.dir/melanoma-cpm.txt'
# matches = 's2-expression_data.dir/melanoma-idmatches.txt'

In [6]:
chdir

<function geode_jupies.chdir(data, sampleclass, genes, gamma=1.0, sort=True, calculate_sig=False, nnull=10, sig_only=False, norm_vector=True)>

In [7]:
# # Infile & outfile
# infiles = ['s1-expression_data.dir/melanoma-cpm.txt']
# outfile = 's1-expression_data.dir/melanoma-IDstatus.txt'

In [8]:
# # Infile & outfile
# infiles = ['rawdata/human_matrix.h5', 'rawdata/m2.csv']
# outfile = 's1-expression_data.dir/melanoma-cpm.txt'

### Test Code

In [9]:
# # Define infiles
# archs_matrix, metadata_file = infiles
# # Read infile metadata_file into pandas
# metadata_df = pd.read_csv(metadata_file)
# # Extract all gsms from the 'id' column
# allgsm = metadata_df['id'].values
# # Save only the unique values by converting the array to a set first, then to a list
# gsm_unique = list(set(allgsm))
# # Read expression data
# expression_dataframe = load_read_counts(archs_matrix, gsms = gsm_unique)
# # Set index name
# expression_dataframe.index.name = 'gene'
# # Perform counts per million normalization
# cpm_table = compute_CPMs(expression_dataframe)

NameError: name 'infiles' is not defined

In [ ]:
# # Read the previously generated infiles
# melanoma_masterdf = pd.read_csv(melanoma_masterfile,index_col='study_index')
# cpm_table = pd.read_table(cpm, index_col='gene')

# # create a dictionary called matches
# matches = {} 
# for std_i, row in melanoma_masterdf.iterrows(): # std_i is the key of the dictionary, index of the dataframe
#     ctrl_gsms = row['ctrl_id'].split(' ')
#     pert_gsms = row['perturb_id'].split(' ')
#     match = np.zeros(cpm_table.shape[1], dtype=np.int32) # Return a new array of given shape and type, filled with zeros.
#     match[np.in1d(cpm_table.columns, ctrl_gsms)] = 1
#     match[np.in1d(cpm_table.columns, pert_gsms)] = 2
#     matches[std_i] = match

# matches_df=pd.DataFrame(matches,index=cpm_table.columns)

In [ ]:
# # Read dataframes
# matches_df=pd.read_table('s2-signatures.dir/melanoma-idmatches.txt', index_col='sample')
# cpm_table=pd.read_table('s1-expression_data.dir/melanoma-cpm.txt', index_col='gene')

# # Initialize results
# results = []

# # Looooop
# for std_i, match in matches_df.items():
#     cd_res = geode_jupies.chdir(cpm_table.values, match.values, cpm_table.index, 
#                     gamma=.5, sort=False, calculate_sig=False)
#     cd_dataframe = pd.DataFrame(cd_res).rename(columns={1: 'gene', 0: 'CD'})
#     cd_dataframe['signature'] = std_i
#     results.append(cd_dataframe)

# # Concatenate and merge
# result_dataframe = pd.concat(results)
# result_table = result_dataframe.pivot(index='gene', columns='signature', values='CD')

# # Write
# result_table.to_csv(outfile, sep='\t')

In [10]:
 # create empty dictionary
dict_top_genes ={}
sig_df = pd.read_table(sig).set_index('gene')

# Get number of studies
n = len(sig_df.columns)
i = 0

# Loop through studies 
for study in sig_df.columns[:]:

    # Print status
    i += 1
    print('Doing study {study} ({i}/{n})...'.format(**locals()))

     # Extract column
    col = sig_df[study].sort_values(ascending=False) # sort values in decreasing order

    genesets = {
        "top":col.index[:300].tolist(),
        "bottom":col.index[-300:].tolist()

        }

    # Extract the top 300 genes from the index
    dict_top_genes[study] = genesets
    
# # open file
# f = open(outfile, 'w')
# # write to file
# f.write(json.dumps(dict_top_genes, ensure_ascii=False, indent=4))     
# # close file
# f.close()

Doing study 1 (1/22)...
Doing study 11 (2/22)...
Doing study 15_a (3/22)...
Doing study 15_b (4/22)...
Doing study 15_c (5/22)...
Doing study 17_a (6/22)...
Doing study 17_b (7/22)...
Doing study 17_c (8/22)...
Doing study 18_a (9/22)...
Doing study 18_b (10/22)...
Doing study 18_c (11/22)...
Doing study 19_a (12/22)...
Doing study 19_b (13/22)...
Doing study 19_c (14/22)...
Doing study 19_d (15/22)...
Doing study 2_a (16/22)...
Doing study 2_b (17/22)...
Doing study 2_c (18/22)...
Doing study 3 (19/22)...
Doing study 5_a (20/22)...
Doing study 5_b (21/22)...
Doing study 5_c (22/22)...


In [11]:
dict_top_genes

{'1': {'top': ['COL1A1',
   'COL5A1',
   'COL6A1',
   'FLNA',
   'MMP2',
   'ACTG1',
   'MT-ND4',
   'LGALS1',
   'COL6A2',
   'LMNA',
   'FTH1',
   'S100A6',
   'HPCAL1',
   'HLA-B',
   'CLU',
   'S100A4',
   'HLA-C',
   'PKM',
   'KRT81',
   'FAM129B',
   'ANPEP',
   'HLA-A',
   'MT-CO1',
   'CCND1',
   'SPARC',
   'AHNAK',
   'SLC16A3',
   'ACTB',
   'AGRN',
   'RPS18',
   'STC2',
   'MT-ND2',
   'C3',
   'ATP2A2',
   'COL7A1',
   'VEGFA',
   'SERPINE2',
   'PLAU',
   'TIMP2',
   'PTPN14',
   'CALM2',
   'SLC7A5',
   'TAGLN2',
   'RPL41',
   'YWHAZ',
   'MT-CYB',
   'MYL6',
   'SQSTM1',
   'TNFRSF21',
   'LGALS3BP',
   'EMILIN1',
   'SLC16A1',
   'COL1A2',
   'POSTN',
   'CSNK1D',
   'PRSS23',
   'ANXA2',
   'ACTN4',
   'MMP14',
   'COL15A1',
   'H1F0',
   'LAMA5',
   'ENO1',
   'ALDOA',
   'UBC',
   'CAPN2',
   'MT-ND1',
   'GAPDH',
   'MYL12A',
   'LRP10',
   'VCL',
   'AKR1B1',
   'LIMK1',
   'PTPRN',
   'SPOCK1',
   'PRKCA',
   'NPTX1',
   'SSH1',
   'PCOLCE',
   'CD74',
   'RPL

In [ ]:
#  # create empty dictionary
#     dict_signatures = {}
#     df_expfpkm_zscore = pd.read_table(infile).set_index('gene_symbol')
    
    
#     # Get number of samples
#     n = len(df_expfpkm_zscore.columns)
#     i = 0
    
#     # Loop through samples 
#     for sample in df_expfpkm_zscore.columns[:]:

#         # Print status
#         i += 1
#         print('Doing sample {sample} ({i}/{n})...'.format(**locals()))

#          # Extract column
#         col = df_expfpkm_zscore[sample].sort_values(ascending=False) # sort values in decreasing order
        
#         genesets = {
#             "top":col.index[:500].tolist(),
#             "bottom":col.index[-500:].tolist()
            
#             }
            
#         # Extract the top 500 genes from the index
#         dict_signatures[sample] = genesets